In [1]:
# python temoa_model/temoa_stochastic.py --config=temoa_model/config_sample (Cluster)
# python temoa_model/temoa_stochastic.py --config=temoa_model/config     (desktop) 
import os
import shutil
from pathlib import Path
import sys
import pandas as pd
import numpy as np

sys.path.insert(0, "../temoa_stochastic/data_processing/")
sys.path.insert(0, "../temoa_stochastic/temoa_model") #temoa model module
import temoa_model
import temoa_config  

#mpi-sppy
import mpisppy.phbase
import mpisppy.opt.ph
import mpisppy.opt.aph
import mpisppy.scenario_tree as scenario_tree
import pyomo.environ as pyo
import mpisppy.utils.sputils as sputils
import sqlite3

#Delete contents of the folder with the input data for TEMOA created in the previous run
def create_or_delete_folder(folder_path):
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
        os.makedirs(folder_path)
        print(f"Contents of folder '{folder_path}' deleted successfully.")
    else:
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created successfully.")
        
#Block Some Prints from TEMOA, too much noise
temp_stdout = None
# Disable
def blockPrint():
    global temp_stdout
    temp_stdout = sys.stdout
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    global temp_stdout
    sys.stdout = temp_stdout


c:\Users\Remote\Desktop\Projects\TemoaHurricane_OEA\TEMOA_GIT\TemoaHurricane_V2\TemoaStochastic\WriteStochasticFiles\../temoa_stochastic/temoa_model\pformat_results.py:673: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if i is '':
c:\Users\Remote\Desktop\Projects\TemoaHurricane_OEA\TEMOA_GIT\TemoaHurricane_V2\TemoaStochastic\WriteStochasticFiles\../temoa_stochastic/temoa_model\pformat_results.py:677: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if i is '':
c:\Users\Remote\Desktop\Projects\TemoaHurricane_OEA\TEMOA_GIT\TemoaHurricane_V2\TemoaStochastic\WriteStochasticFiles\../temoa_stochastic/temoa_model\pformat_results.py:683: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if i is '':
c:\Users\Remote\Desktop\Projects\TemoaHurricane_OEA\TEMOA_GIT\TemoaHurricane_V2\TemoaStochastic\WriteStochasticFiles\../temoa_stochastic/temoa_model\pformat_results.py:689: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if i is '':
c:\Users\Remote\Desktop\Projects

<IPython.core.display.Javascript object>

[    0.00] Initializing mpi-sppy


In [2]:
PowerSystemFileName="NC_EnergySystem2023.sqlite"
PathPowerSystemData="./InputData/"+PowerSystemFileName
PercentageDamagePath="./InputData/PercentageHurricaneDamage.xlsx"
ProbabilityScenariosPath="./InputData/ProbabilityEachScenario.xlsx"
AverageNumberOfEventsPerScenario="./InputData/AverageNumberOfEventsPerScenario.xlsx"

PathWriteTemoaInputData="./Import2TemoaFiles/"
PathScenarioTree=PathWriteTemoaInputData+"Scenarios/"


#Delete contents of the folder with the input data for TEMOA created in the previous run
create_or_delete_folder(PathWriteTemoaInputData)
shutil.copyfile(PathPowerSystemData, PathWriteTemoaInputData+PowerSystemFileName)#copy of .sqlite file

df_percentage_damage = pd.read_excel(PercentageDamagePath)
df_probability_scenarios = pd.read_excel(ProbabilityScenariosPath)
df_num_events_scenario = pd.read_excel(AverageNumberOfEventsPerScenario)
os.makedirs(PathScenarioTree)

Contents of folder './Import2TemoaFiles/' deleted successfully.


# Write Scenario Tree Files

In [3]:
#Create a .dat file from the.sqlite file
DatFilePath=PathScenarioTree+"R_.dat"#Root node

class C_Options:
  def __init__(self, myopic=False, mga_weight=False):
    self.myopic = myopic
    self.mga_weight = mga_weight

options=C_Options()
blockPrint()
temoa_config.db_2_dat(PathPowerSystemData,DatFilePath,options)
enablePrint()
shutil.copyfile(PathPowerSystemData, PathPowerSystemData[:-7]+".dat")#copy of .sqlite file

#Create a pyomo isntance of TEMOA
TEMOA_Model=temoa_model.temoa_create_model()
blockPrint()
Instance = TEMOA_Model.create_instance( DatFilePath ) 
enablePrint()

TimeStages = sorted( getattr(Instance, 'time_optimize').data() )


## Create Nodes

In [4]:
# TEMOA_CAPREDUCTION_Keys= list(Instance.CapReduction.keys())
# dir_CAPREDUCTION_Keys={ "Regions": np.array([i[0] for i in TEMOA_CAPREDUCTION_Keys]),
#                         "Periods": np.array([i[1] for i in TEMOA_CAPREDUCTION_Keys]),
#                         "Tech":    np.array([i[2] for i in TEMOA_CAPREDUCTION_Keys]),
#                         "Vintage": np.array([i[3] for i in TEMOA_CAPREDUCTION_Keys]),
#                         "Key":     TEMOA_CAPREDUCTION_Keys}

#Since I represented all costs even if they are zero, I can use CostFixed to get valid indexed for capacity reduction
con = sqlite3.connect(PathPowerSystemData)
ValidIdxForCapReduction_sql = pd.read_sql_query("SELECT * FROM CostFixed", con)
con.close()



dir_CAPREDUCTION_Keys={ "Regions": ValidIdxForCapReduction_sql["regions"].values,
                        "Periods": ValidIdxForCapReduction_sql["periods"].values,
                        "Tech":    ValidIdxForCapReduction_sql["tech"].values,
                        "Vintage": ValidIdxForCapReduction_sql["vintage"].values}


def WriteNodeData(NodeName, CapReductionNodeInfo, PathScenarioTree=PathScenarioTree, dir_CAPREDUCTION_Keys=dir_CAPREDUCTION_Keys):

    NodeYear=CapReductionNodeInfo.columns[2]
    PreviousNodeYear=CapReductionNodeInfo.columns[3]

    LastYear=CapReductionNodeInfo.columns[-1]

    TEMOA_Region=dir_CAPREDUCTION_Keys["Regions"]
    TEMOA_Period=dir_CAPREDUCTION_Keys["Periods"]
    TEMOA_Tech=dir_CAPREDUCTION_Keys["Tech"]
    TEMOA_Vintage=dir_CAPREDUCTION_Keys["Vintage"]

    file = open(PathScenarioTree+NodeName+".dat", "w")
    Line1="param CapReduction:=\n"
    file.write(Line1)

    for row in CapReductionNodeInfo.iterrows():
        region=row[1]["regions"]
        tech=row[1]["tech"]


        #look at all tech with vintage less than previous node year, those are affected by the cap reduction
        IdxIn2Change=(TEMOA_Region==region)*(TEMOA_Tech==tech)*(TEMOA_Vintage<=PreviousNodeYear)*(TEMOA_Period==NodeYear)#All other elements must be equal to one which is the default value
        #(TEMOA_Period==NodeYear): technology may have already retired at the period of the node

        TEMOA_Region_tmp=TEMOA_Region[IdxIn2Change]
        TEMOA_Period_tmp=TEMOA_Period[IdxIn2Change]
        TEMOA_Tech_tmp=TEMOA_Tech[IdxIn2Change]
        TEMOA_Vintage_tmp=TEMOA_Vintage[IdxIn2Change]
        rate=np.ones(len(TEMOA_Region_tmp))

        for v in np.unique(TEMOA_Vintage_tmp):

            if v<=LastYear:
                IdxIn=(TEMOA_Vintage_tmp<=LastYear)
                rate[IdxIn]=row[1][LastYear]
            else:
                IdxIn=(TEMOA_Vintage_tmp==v)
                rate[IdxIn]=row[1][v]
            
        Lines=[]
        for i in range(len(TEMOA_Region_tmp)):
            Line=TEMOA_Region_tmp[i] +"   "+ str(NodeYear)+"   "+TEMOA_Tech_tmp[i]+"   "+str(TEMOA_Vintage_tmp[i])+"   "+str(rate[i])+"\n"
            file.write(Line)

    file.write(";")
    file.close()
    #print(f"File {NodeName}.dat created successfully.")


In [5]:
NodeStages=TimeStages
NumStages=len(NodeStages)
ScenariosEachStage=list(df_probability_scenarios.columns[1:])
RootNodeName="R_"

LastStageNodesMapped=[RootNodeName]
LastStageNodesMapped_tmp=[]
CountTimeStages=1

SetNodes=[]
SetNodes.append(RootNodeName)
Dic_ChildrenNodes={}

Dic_NodeYear={}
Dic_NodeYear[RootNodeName]=NodeStages[0]

ConditionalProbability={}
ConditionalProbability[RootNodeName]=1

while CountTimeStages<NumStages:
    print("Stage: %d ----ok"%(CountTimeStages))
    for LSNode in LastStageNodesMapped:
        ListChildrenNodes=[]
        for scenario in ScenariosEachStage:
            
            NodeName=LSNode+scenario+"_"
            
            LastStageNodesMapped_tmp.append(NodeName)
            ScenariosNeeded=NodeName.split("_")[1:-1]#Sequence of Nodes
            ScenariosNeeded.reverse()

            #Put together the information of the nodes needed to create the scenario
            Vintages=NodeStages[0:CountTimeStages]#Vintage Intervals we need to be aware of
            Vintages.reverse()

            
            #Contain information about capacity reduction for each technology and region Considering its vintage at the current node
            #Which is associated with a CurrentTimeStage
            CapReductionNodeInfo=df_percentage_damage[["regions","tech"]].copy() 
            CurrentTimeStage=NodeStages[CountTimeStages]
            CapReductionNodeInfo[CurrentTimeStage]=1 #Tech deployed (vintage) at the node time stage is not affected by damages at of before the node
            
            #Mapping different tech vintages from the perspective of the current node
            #What is the damage a tech with vintage (X) see at the current node?
            for i in range(len(Vintages)):
                Vintage=Vintages[i] #Remember vintages are reversed
                s_tmp=ScenariosNeeded[0:i+1] #Scenarios associated with the vintage (are reversed)
                
                CapReductionNodeInfo[Vintage]=1 #Capacity reduction for technologies deployed before this specific vintage (will be updated)

                count_runs=0
                for s in s_tmp:
                    year_tmp_scenario=Vintages[count_runs]#Year of the individual S$ scenario
                    count_runs=count_runs+1
                    
                    AvgNumEventsInScenario=df_num_events_scenario.loc[df_num_events_scenario["t_periods"]==year_tmp_scenario,s].values[0]# Depends on the length of the period
                    P_DamageNodeInfo=df_percentage_damage[s]*AvgNumEventsInScenario
                    
                    if s=="S0":
                        #For the case where we have S0 we need to adjust the damage to account for no damage events
                        #The original S0 from excel assumes that at least one hazus recurrent event happens
                        #New Damage = PDamage(H)*P(H)+ 0*(1-P(H))=PDamage(H)*P(H)
                        P_Hazus=df_probability_scenarios.loc[df_probability_scenarios["t_periods"]==year_tmp_scenario,s].values[0]
                        P_DamageNodeInfo=P_DamageNodeInfo*P_Hazus #Will reduce the damage to take into account no hurricane may happen 
                    
                    CapReductionNodeInfo[Vintage]=CapReductionNodeInfo[Vintage]*(1-P_DamageNodeInfo)

            LastScenario=ScenariosNeeded[0]# Remember array is reversed
            
            if LastScenario!="S0":
                ConditionalProbability[NodeName]=round(df_probability_scenarios.at[len(ScenariosNeeded)-1,LastScenario],10)
            else:
                #The probabilities on excel consider that at least one hazus event happens
                #we need to adjust the probabilities to account for the case where no hazus event happens
                # P(S0)=1-P(S1)-P(S2)-P(S3)-P(S4),...-P(Sn)
                ConditionalProbability[NodeName]=round(1-df_probability_scenarios.iloc[len(ScenariosNeeded)-1,2:].sum(),10)
                
            ListChildrenNodes.append(NodeName)
            SetNodes.append(NodeName)
            WriteNodeData(NodeName, CapReductionNodeInfo)
            Dic_NodeYear[NodeName]=NodeStages[CountTimeStages]


        Dic_ChildrenNodes[LSNode]=ListChildrenNodes

    CountTimeStages=CountTimeStages+1        
    LastStageNodesMapped=LastStageNodesMapped_tmp
    LastStageNodesMapped_tmp=[]

Stage: 1 ----ok
Stage: 2 ----ok
Stage: 3 ----ok
Stage: 4 ----ok
Stage: 5 ----ok
Stage: 6 ----ok


## Create Tree

In [ ]:
Scenarios=[ '_'.join(s.split("_")[1:-1]) for s in LastStageNodesMapped] #Create scenario names

#Write satage name of each node
Dic_NodeStage={}
for iten in list(Dic_NodeYear.keys()):
    Dic_NodeStage[iten]="s"+str(Dic_NodeYear[iten])

#Write ScenarioLeafNode
ScenarioLeafNode={}
for s in Scenarios:
    ScenarioLeafNode[s]=RootNodeName+s+"_"


In [ ]:
#Stage Variables

def StageVariablesList(Period,Instance=Instance):
    stage_vars = list()

    V_FlowOut_keys = [index for index in Instance.V_FlowOut.keys()
                    if index[1] == Period]
    stage_vars.extend(sorted(set('V_FlowOut[{},{},{},{},{},{},{},{}]'.format( *index )
                for index in V_FlowOut_keys )))


    V_FlowIn_keys = [index for index in Instance.V_FlowIn.keys()
                    if index[1] == Period]
    stage_vars.extend(sorted(set('V_FlowIn[{},{},{},{},{},{},{},{}]'.format( *index )
                for index in V_FlowIn_keys )))


    V_FlowOutAnnual_keys= [index for index in Instance.V_FlowOutAnnual.keys()
                        if index[1] == Period]
    stage_vars.extend(sorted(set('V_FlowOutAnnual[{},{},{},{},{},{}]'.format( *index )
                for index in V_FlowOutAnnual_keys )))


    V_Flex_keys = [index for index in Instance.V_Flex.keys()
                        if index[1] == Period]
    stage_vars.extend(sorted(set('V_Flex[{},{},{},{},{},{},{},{}]'.format( *index )
                for index in V_Flex_keys )))


    V_FlexAnnual_keys = [index for index in Instance.V_FlexAnnual.keys()
                        if index[1] == Period]
    stage_vars.extend(sorted(set('V_FlexAnnual[{},{},{},{},{},{}]'.format( *index )
                for index in V_FlexAnnual_keys )))


    V_Curtailment_keys = [index for index in Instance.V_Curtailment.keys()
                        if index[1] == Period]
    stage_vars.extend(sorted(set('V_Curtailment[{},{},{},{},{},{},{},{}]'.format( *index )
                for index in V_Curtailment_keys )))


    V_StorageLevel_keys = [index for index in Instance.V_StorageLevel.keys()
                        if index[1] == Period]
    stage_vars.extend(sorted(set('V_StorageLevel[{},{},{},{},{},{}]'.format( *index )
                for index in V_StorageLevel_keys )))


    V_StorageInit_keys = [index for index in Instance.V_StorageInit.keys()
                        if index[1] == Period]
    stage_vars.extend(sorted(set('V_StorageInit[{},{},{},{},{},{}]'.format( *index )
                for index in V_StorageInit_keys )))


    V_Capacity_keys = [index for index in Instance.V_Capacity.keys()
                        if index[1] == Period]
    stage_vars.extend(sorted(set('V_Capacity[{},{},{}]'.format( *index )
                for index in V_Capacity_keys )))


    V_CapacityAvailableByPeriodAndTech_keys = [index for index in Instance.V_CapacityAvailableByPeriodAndTech.keys()
                        if index[1] == Period]
    stage_vars.extend(sorted(set('V_CapacityAvailableByPeriodAndTech[{},{},{}]'.format( *index )
                for index in V_CapacityAvailableByPeriodAndTech_keys )))
    return stage_vars



In [ ]:
#def WriteTree(NodeStages, PathScenarioTree=PathScenarioTree):
file = open(PathScenarioTree+"ScenarioStructure.dat", "w")

#Stages
file.write("set  Stages  :=\n")
for s in NodeStages:
    Line="s"+str(s)+"\n"
    file.write(Line)
file.write(";\n")

#Scenarios
file.write("set  Scenarios  :=\n")
for s in Scenarios:
    file.write(s+"\n")
file.write(";\n")

#Nodes
file.write("set  Nodes  :=\n")
for n in SetNodes:
    file.write(n+"\n")
file.write(";\n")

#Children Nodes
for MainNode in list(Dic_ChildrenNodes.keys()):
    ChNodes=Dic_ChildrenNodes[MainNode]
    file.write("set  Children["+MainNode+"]  :=\n")
    for n in ChNodes:
        file.write(n+"\n")
    file.write(";\n")

#Node Stage
file.write("param  NodeStage  :=\n")
for n in list(Dic_NodeStage.keys()):
    Line=n+"   "+Dic_NodeStage[n]+"\n"
    file.write(Line)
file.write(";\n")

#ScenarioLeafNode
file.write("param  ScenarioLeafNode  :=\n")
for s in list(ScenarioLeafNode.keys()):
    Line=s+"   "+ScenarioLeafNode[s]+"\n"
    file.write(Line)
file.write(";\n")

#StageCost
file.write("param  StageCost  :=\n")
for sy in NodeStages:
    Line="s"+str(sy)+"   "+"StochasticPointCost["+str(sy)+"]\n"
    file.write(Line)

file.write(";\n")

#ConditionalProbability
file.write("param  ConditionalProbability  :=\n")
for n in list(ConditionalProbability.keys()):
    Line=n+"   "+str(ConditionalProbability[n])+"\n"
    file.write(Line)

file.write(";\n")

#Stage Variables
for period in NodeStages:
    stage_vars=StageVariablesList(Period=period)

    file.write("set  StageVariables[s"+str(period)+"]"+"  :=\n")
    for v in stage_vars:
        file.write(v+"\n")
    file.write(";\n")

#ScenarioBasedData
file.write("param  ScenarioBasedData  :=  False ;")

file.close()